<a href="https://colab.research.google.com/github/A-Istrate/80629_Project/blob/main/ML_TPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [266]:
import numpy as np
import pandas as pd
import math
import copy
from random import seed
from random import random
import matplotlib.pyplot as plt  # import matplotlib for plotting and visualization
import matplotlib
import PIL
from PIL import Image
import PIL.Image
import tensorflow as tf
import pathlib
import os
import re
import gc
import sklearn
from tensorflow.keras import layers
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks 

-------------
Paths

In [2]:
### Train Validation TEST and survey
path_train = pathlib.Path(r"C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers_final\train")
path_valid = pathlib.Path(r"C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers_final\valid")
path_test = pathlib.Path(r"C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers_final\test")
path_survey = pathlib.Path(r"C:\Users\istra\Google Drive\Cours HEC\Maitrise\Semestre 2\Machine_Learning\TP GROUPE\book_covers_final\survey")

Loading Models

In [38]:
## Loading NLP Model 1
NLP_model_1 = keras.models.load_model("NLP_LSTM1.h5")

In [41]:
## Loading NLP Model 
NLP_model_2 = keras.models.load_model("NLP_LSTM1_nw.h5")

In [5]:
## Loading Resnet Model 1
import tensorflow_hub as hub
CNN_model_1 = keras.models.load_model("resnet_nd_15.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [6]:
## Loading VGG Model 2
import tensorflow_hub as hub
CNN_model_2 = keras.models.load_model("model_VGG2.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [7]:
## Loading CNN Model 3
import tensorflow_hub as hub
CNN_model_3 = keras.models.load_model("model_ns_nd.h5",custom_objects={'KerasLayer':hub.KerasLayer})

----------- 
Data set up

In [8]:
## CSV files
train=pd.read_csv('train.csv') 
valid=pd.read_csv('valid.csv')   
test=pd.read_csv('test.csv')  
survey=pd.read_csv('survey.csv')

In [10]:
## Image parameters
batch_size =128
img_height = 150
img_width =100

In [11]:
## train
list_train = tf.data.Dataset.list_files(str(str(path_train)+"\*/*.jp*"),shuffle=False)
list_train = list_train.shuffle(len(list_train),reshuffle_each_iteration=False)
## valid
list_valid = tf.data.Dataset.list_files(str(str(path_valid)+"\*/*.jp*"),shuffle=False)
list_valid = list_valid.shuffle(len(list_valid),reshuffle_each_iteration=False)
## test
list_test = tf.data.Dataset.list_files(str(str(path_test)+"\*/*.jp*"),shuffle=False)
list_test = list_test.shuffle(len(list_test),reshuffle_each_iteration=False)
## survey
list_survey = tf.data.Dataset.list_files(str(str(path_survey)+"\*/*.jp*"),shuffle=False)
list_survey = list_survey.shuffle(len(list_survey),reshuffle_each_iteration=False)

In [144]:
# getting the classes names
class_names = np.array(sorted([item.name for item in path_train.glob('*') if os.path.isdir(item)]))
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])
def process_path(file_path):
    label = get_label(file_path)
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
         
    return img,label

In [185]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
AUTOTUNE = tf.data.AUTOTUNE
train_ds = list_train.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = list_valid.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = list_test.map(process_path, num_parallel_calls=AUTOTUNE)
survey_ds = list_survey.map(process_path, num_parallel_calls=AUTOTUNE)

In [187]:
def configure_for_performance_train(ds):
  ds = ds.cache()
  #ds = ds.shuffle(buffer_size=1000,reshuffle_each_iteration=True)
  #print(ds)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

def configure_for_performance_other(ds):
  ds = ds.cache()
  # ds = ds.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = configure_for_performance_train(train_ds)
val_ds = configure_for_performance_other(val_ds)
test_ds = configure_for_performance_other(test_ds)
survey_ds = configure_for_performance_other(survey_ds)

In [16]:
## getting the images names and their order 
train_images = [str(obj.numpy()).rsplit("\\")[-1][:-1] for obj in list_train]
valid_images = [str(obj.numpy()).rsplit("\\")[-1][:-1] for obj in list_valid]
test_images = [str(obj.numpy()).rsplit("\\")[-1][:-1] for obj in list_test]
survey_images = [str(obj.numpy()).rsplit("_")[-1][:-1] for obj in list_survey]

In [17]:
# reindexing train csv 
train = train.set_index("img")
train = train.reindex(train_images)
# reindexing valid csv 
valid = valid.set_index("img")
valid = valid.reindex(valid_images)
# reindexing test csv 
test = test.set_index("img")
test = test.reindex(test_images)
# reindexing survey csv 
survey = survey.set_index("img")
survey = survey.reindex(survey_images)

In [18]:
#Checking if the train data sets are in the same order  
[[a,b] for a,b in zip(train.index[1:10],train_images[1:10])]

[['15365.jpg', '15365.jpg'],
 ['40567.jpg', '40567.jpg'],
 ['50757.jpg', '50757.jpg'],
 ['20475.jpg', '20475.jpg'],
 ['9641.jpg', '9641.jpg'],
 ['51189.jpg', '51189.jpg'],
 ['14400.jpg', '14400.jpg'],
 ['31730.jpg', '31730.jpg'],
 ['24676.jpg', '24676.jpg']]

In [19]:
#Checking if the test data sets are in the same order  
[[a,b] for a,b in zip(test.index[1:10],test_images[1:10])]

[['23288.jpg', '23288.jpg'],
 ['4894.jpg', '4894.jpg'],
 ['26250.jpg', '26250.jpg'],
 ['6997.jpg', '6997.jpg'],
 ['26368.jpg', '26368.jpg'],
 ['54068.jpg', '54068.jpg'],
 ['43044.jpg', '43044.jpg'],
 ['51084.jpg', '51084.jpg'],
 ['40610.jpg', '40610.jpg']]

-------
Clean Titles
def clean_titles():


In [20]:
#Transforming the book title to a list 
def clean_titles(string):
    string = re.sub("[^a-zA-Z\s]","",string).lower().strip()
    splited=string.split(" ")
    if len(splited)>1:
        return splited
    else:
        return ""
# appling to all 4 data frames 
train["title"] = train.book_title.map(clean_titles)
valid["title"] = valid.book_title.map(clean_titles)
test["title"] = test.book_title.map(clean_titles)
survey["title"] = survey.book_title.map(clean_titles)

In [22]:
all=pd.concat([test.title,train.title,valid.title,survey.title])
xtoken = Tokenizer(oov_token=True)
xtoken.fit_on_texts(all)
#prepare title
def prep_title(title):
    
    if len(title)>1:
        x_val = xtoken.texts_to_sequences(title)
        x_val = pad_sequences(x_val, maxlen=41, padding='pre')
    else:
        x_val = np.zeros(41)
    return x_val

In [23]:
exp_token = xtoken
%store exp_token

Stored 'exp_token' (Tokenizer)


In [24]:
a_test = prep_title(test.title)
a_train = prep_title(train.title)
a_valid = prep_title(valid.title)
a_survey =prep_title(survey.title)

In [25]:
a_survey[22]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           2,   348,    27,     2, 22744])

-------------------------------
Naive predictions

In [308]:
from langdetect import detect#remove list that are not in english
def detect2(s):
    if len(s)>2:
        try:
            d = detect(s)
        except:
            d = 'na'
        return d
    else:
        return 'na'

In [304]:
filtered_test = test[test.title.map(" ".join).map(detect2)=="en"]

In [195]:
%store -r x_test
%store -r y_test

In [39]:
predictions_NLP = NLP_model_1.predict(a_test)

In [43]:
predictions_NLP_2 = NLP_model_2.predict(a_test)

In [28]:
test.title[0]

['betsy', 'in', 'spite', 'of', 'herself']

In [29]:
a = np.asarray(prep_title(test.title))
a[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,   13, 9523, 1968])

In [45]:
predictions_CNN = CNN_model_1.predict(test_ds)

In [46]:
predictions_CNN_2 = CNN_model_2.predict(test_ds)

In [47]:
predictions_CNN_3 = CNN_model_3.predict(test_ds)

In [33]:
def prediction_score(valid,predictions,cnames=class_names):
    pred = np.argmax(predictions, axis=1)
    pred2 = np.argsort(predictions,axis=1)[:,-2]
    pred3 = np.argsort(predictions,axis=1)[:,-3]
    dict_c = dict(enumerate(cnames))
    scores=[]
    for v,p1,p2,p3 in zip(valid,pred,pred2,pred3):
        score=0
        top1 = 0
        top3 = 0
        if v == p1: score=1
        elif v == p2: score =0.75
        elif v == p3: score =0.5
        if score == 1 : top1 =1
        if score >=0.5 : top3=1
        scores.append([dict_c.get(v),score,top1,top3])
    scores_df = pd.DataFrame(scores,columns=["genre","score","top1","top3"])
    meanscore =np.round(scores_df["score"].mean(),3)
    sumtop = np.sum(scores_df["top1"].sum())
    print("The average score is %s" %meanscore)
    print("Correctly predicted %s / %s" % (sumtop,len(scores_df)))
    print(scores_df.groupby("genre").agg({"score":"mean","top1":["sum","mean"],"top3":["sum","mean"],"genre":"count"}).round(2))
    return 

In [48]:
predictions = predictions_NLP +predictions_NLP_2+predictions_CNN +predictions_CNN_2+predictions_CNN_3
predicted_classes = np.argmax(predictions, axis=1)
#oclasses = [[c.numpy() for c in object[1]]for object in test_ds]
#original_classes = np.concatenate(oclasses).ravel()
original_classes=test.main_label.map(list(class_names).index) #np.argmax(y_test,axis=1) #
print(classification_report(original_classes, predicted_classes, target_names=class_names,zero_division=0))
prediction_score(original_classes,predictions)

                       precision    recall  f1-score   support

            Biography       0.55      0.43      0.48       169
            Childrens       0.56      0.41      0.48       140
              Fantasy       0.48      0.73      0.58       749
           Historical       0.38      0.38      0.38       383
               Horror       0.49      0.20      0.29       113
      Mystery & Crime       0.43      0.44      0.43       341
               Poetry       0.47      0.24      0.32        96
   Politics & History       0.53      0.41      0.46       189
Religion & Philosophy       0.56      0.50      0.53       262
              Romance       0.50      0.59      0.54       557
      Science Fiction       0.41      0.26      0.32       221
       Sequential Art       0.71      0.58      0.64       138
          Young Adult       0.42      0.27      0.33       388

             accuracy                           0.48      3746
            macro avg       0.50      0.42      0.44 

------------------------------------------
Complex fusion of the models

In [233]:
all_models =list()
all_models=[NLP_model_1,NLP_model_2,CNN_model_1,CNN_model_2,CNN_model_3]
NLP_models=[NLP_model_1,NLP_model_2]
CNN_models=[CNN_model_1,CNN_model_2,CNN_model_3]
mix_models=[NLP_model_1,CNN_model_1]

In [75]:
NLP_model_1.input_shape

(None, 41)

In [103]:
# define stacked model from multiple member input models
def define_stacked_model(members):
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
	# define multi-headed input
	#inputA = Input(shape=(None, 41))
	#inputB = Input(shape=(None, 150, 100, 3))
	#members[0](inputA)
	#members[1](inputA)
	#members[2](inputB)
	#members[3](inputB)
	#members[4](inputB)
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	hidden = Dense(65, activation='relu')(merge)
	output = Dense(13, activation='softmax')(hidden)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [108]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	print(np.shape(X))
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	model.fit(X, inputy_enc, epochs=10, verbose=0)

In [52]:
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [53]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin/'

In [134]:
from keras.utils import plot_model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
stacked_model = define_stacked_model(all_models)
NLP_stacked_model = define_stacked_model(NLP_models)
CNN_stacked_model = define_stacked_model(CNN_models)


In [80]:
original_train_classes=train.main_label.map(list(class_names).index)

In [93]:
X = [a_train,a_train,train_ds,train_ds,train_ds]

In [101]:
from keras.utils import to_categorical
Y = np.asarray(original_train_classes)

In [209]:
Y = to_categorical(Y)

In [114]:
stacked_model.input

[<KerasTensor: shape=(None, 41) dtype=float32 (created by layer 'embedding_2_input')>,
 <KerasTensor: shape=(None, 41) dtype=float32 (created by layer 'embedding_3_input')>,
 <KerasTensor: shape=(None, 150, 100, 3) dtype=float32 (created by layer 'rescaling_1_input')>,
 <KerasTensor: shape=(None, 150, 100, 3) dtype=float32 (created by layer 'rescaling_3_input')>,
 <KerasTensor: shape=(None, 150, 100, 3) dtype=float32 (created by layer 'rescaling_6_input')>]

In [ ]:
fit_stacked_model(stacked_model,X,Y)

In [122]:
NLP_stacked_model.fit([a_train,a_train],Y,epochs=10)

Epoch 1/10
990/990 [==============================] - 11s 5ms/step - loss: 1.5243 - accuracy: 0.5892
Epoch 2/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9327 - accuracy: 0.7116
Epoch 3/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9297 - accuracy: 0.7139
Epoch 4/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9046 - accuracy: 0.7181
Epoch 5/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9157 - accuracy: 0.7175
Epoch 6/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9167 - accuracy: 0.7158
Epoch 7/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9126 - accuracy: 0.7169
Epoch 8/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9128 - accuracy: 0.7153
Epoch 9/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9252 - accuracy: 0.7144
Epoch 10/10
990/990 [==============================] - 5s 5ms/step - loss: 0.9160 - accuracy: 0.717

In [126]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [228]:
def define_stacked_model_CNN(members):
# update all layers in all models to not be trainable
 for i in range(len(members)):
  model = members[i]
  for layer in model.layers:
   # make not trainable
   layer.trainable = False
# rename to avoid 'unique layer name' issue
   layer._name = 'ensemble_' + str(i+1) + '_' + layer.name

#inputB = Input(shape=(None, 150, 100, 3))
 #inputA = Input(shape=(150,100,3,))
# members[0](inputA)
# members[1](inputA)
# members[2](inputA)
#members[2](inputB)
#members[3](inputB)
#members[4](inputB)
 ensemble_visible = [model.input for model in members]
# concatenate merge output from each model
#ensemble_outputs = [model.output for model in members]
 merge = concatenate(axis=1)([members[0].output,members[1].output,members[2].output])
 hidden = Dense(65, activation='relu')(merge)
 S_output = Dense(13, activation='softmax')(hidden)
 model = Model(inputs=ensemble_visible, outputs=S_output)
# plot graph of ensemble
 plot_model(model, show_shapes=True, to_file='model_graph.png')
# compile
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [151]:
CNN_model_1.input_shape

(None, 150, 100, 3)

In [235]:
X=[train,train_ds]

 

In [234]:
stacked_mix_models = define_stacked_model(mix_models)

In [229]:
CNN_stacked_model = define_stacked_model(CNN_models)

In [241]:
stacked_mix_models.fit(X,Y,epochs=1)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'pandas.core.frame.DataFrame'>", "<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>"}), <class 'numpy.ndarray'>

--------------------------------------
Other try

In [244]:
from numpy import dstack
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [245]:
stack_NLP = stacked_dataset(NLP_models,a_train)

In [267]:
stack_NLP_valid = stacked_dataset(NLP_models,a_valid)
stack_NLP_test = stacked_dataset(NLP_models,a_test)
stack_NLP_survey = stacked_dataset(NLP_models,a_survey)

In [246]:
stack_CNN = stacked_dataset(CNN_models,train_ds)

In [268]:
stack_CNN_valid = stacked_dataset(CNN_models,val_ds)
stack_CNN_test = stacked_dataset(CNN_models,test_ds)
stack_CNN_survey = stacked_dataset(CNN_models,survey_ds)

In [247]:
import pickle

In [248]:
with open('cnn_predict.pickle', 'wb') as output:
    pickle.dump(stack_CNN, output)

In [269]:
with open('stack_CNN_valid.pickle', 'wb') as output:
    pickle.dump(stack_CNN_valid, output)
with open('stack_CNN_test.pickle', 'wb') as output:
    pickle.dump(stack_CNN_test, output)
with open('stack_CNN_survey.pickle', 'wb') as output:
    pickle.dump(stack_CNN_survey, output)

In [254]:
#merging both data sets
train_predict = np.concatenate((stack_NLP,stack_CNN),axis=1)

In [270]:
valid_predict = np.concatenate((stack_NLP_valid,stack_CNN_valid),axis=1)
test_predict = np.concatenate((stack_NLP_test,stack_CNN_test),axis=1)
survey_predict = np.concatenate((stack_NLP_survey,stack_CNN_survey),axis=1)

In [255]:
with open('train_predict.pickle', 'wb') as output:
    pickle.dump(train_predict, output)

In [462]:
from keras.regularizers import l2
model_stacked = tf.keras.Sequential([
   layers.Dropout(0.6), 
   layers.Dense(13, activation='softmax', activity_regularizer=l2(0.1))
])
model_stacked.compile(
   
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0002,
        
    ),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
ES= callbacks.EarlyStopping(monitor='val_accuracy', patience=2)

In [271]:
Y_train = np.asarray(original_train_classes)
Y_valid = np.asarray(valid.main_label.map(list(class_names).index))
Y_test = np.asarray(test.main_label.map(list(class_names).index))
Y_survey = np.asarray(survey.main_label.map(list(class_names).index))

In [465]:
history_stacked = model_stacked.fit(train_predict,Y_train,validation_data=(valid_predict, Y_valid),callbacks=[ES],epochs=45)

Epoch 1/45
990/990 [==============================] - 1s 927us/step - loss: 1.0533 - accuracy: 0.6839 - val_loss: 1.6301 - val_accuracy: 0.4980
Epoch 2/45
990/990 [==============================] - 1s 878us/step - loss: 1.0569 - accuracy: 0.6831 - val_loss: 1.6302 - val_accuracy: 0.4977
Epoch 3/45
990/990 [==============================] - 1s 886us/step - loss: 1.0401 - accuracy: 0.6892 - val_loss: 1.6308 - val_accuracy: 0.4984
Epoch 4/45
990/990 [==============================] - 1s 903us/step - loss: 1.0580 - accuracy: 0.6857 - val_loss: 1.6312 - val_accuracy: 0.4976
Epoch 5/45
990/990 [==============================] - 1s 985us/step - loss: 1.0556 - accuracy: 0.6859 - val_loss: 1.6316 - val_accuracy: 0.4985
Epoch 6/45
990/990 [==============================] - 1s 884us/step - loss: 1.0531 - accuracy: 0.6862 - val_loss: 1.6318 - val_accuracy: 0.4985
Epoch 7/45
990/990 [==============================] - 1s 880us/step - loss: 1.0396 - accuracy: 0.6898 - val_loss: 1.6323 - val_accuracy:

In [467]:
predictions = model_stacked.predict(test_predict)

In [468]:
predicted_classes = np.argmax(predictions, axis=1)
original_classes=test.main_label.map(list(class_names).index) 
print(classification_report(original_classes, predicted_classes, target_names=class_names,zero_division=0))
prediction_score(original_classes,predictions)

                       precision    recall  f1-score   support

            Biography       0.51      0.43      0.47       169
            Childrens       0.62      0.42      0.50       140
              Fantasy       0.52      0.69      0.60       749
           Historical       0.34      0.43      0.38       383
               Horror       0.53      0.18      0.26       113
      Mystery & Crime       0.45      0.46      0.46       341
               Poetry       0.65      0.23      0.34        96
   Politics & History       0.61      0.38      0.47       189
Religion & Philosophy       0.49      0.51      0.50       262
              Romance       0.54      0.58      0.56       557
      Science Fiction       0.38      0.26      0.31       221
       Sequential Art       0.76      0.63      0.69       138
          Young Adult       0.38      0.35      0.36       388

             accuracy                           0.49      3746
            macro avg       0.52      0.43      0.45 

In [470]:
survey_predictions = model_stacked.predict(survey_predict)
predicted_classes = np.argmax(survey_predictions, axis=1)
original_classes=survey.main_label.map(list(class_names).index) 
print(classification_report(original_classes, predicted_classes, target_names=class_names,zero_division=0))
prediction_score(original_classes,survey_predictions)

                       precision    recall  f1-score   support

            Biography       0.75      0.60      0.67         5
            Childrens       1.00      0.60      0.75         5
              Fantasy       0.27      0.80      0.40         5
           Historical       0.33      0.60      0.43         5
               Horror       0.00      0.00      0.00         5
      Mystery & Crime       0.50      0.20      0.29         5
               Poetry       0.50      0.20      0.29         5
   Politics & History       1.00      0.20      0.33         5
Religion & Philosophy       0.44      0.80      0.57         5
              Romance       0.62      1.00      0.77         5
      Science Fiction       0.50      0.40      0.44         5
       Sequential Art       1.00      0.80      0.89         5
          Young Adult       0.67      0.40      0.50         5

             accuracy                           0.51        65
            macro avg       0.58      0.51      0.49 

In [472]:
from sklearn.utils import class_weight

   

#cw = class_weight.compute_class_weight('balanced',np.unique(books['main_label']), books['main_label']),



cw = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
cw = dict(enumerate(cw))

C:\Users\istra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12], y=[ 7  9  4 ...  8 10  7] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [471]:
Y_train

array([ 7,  9,  4, ...,  8, 10,  7], dtype=int64)

In [510]:
model_stacked_w = tf.keras.Sequential([
    
    layers.Dropout(0.6),
   layers.Dense(13, activation='softmax', activity_regularizer=l2(0.25))
])
model_stacked_w.compile(
   
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.002,
        
    ),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
ES= callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [511]:
history_stacked_weighted = model_stacked_w.fit(train_predict,Y_train,validation_data=(valid_predict, Y_valid),callbacks=[ES],epochs=45,class_weight=cw)

Epoch 1/45
990/990 [==============================] - 1s 1ms/step - loss: 2.0202 - accuracy: 0.4528 - val_loss: 1.8694 - val_accuracy: 0.4760
Epoch 2/45
990/990 [==============================] - 1s 921us/step - loss: 1.3038 - accuracy: 0.6638 - val_loss: 1.7842 - val_accuracy: 0.4736
Epoch 3/45
990/990 [==============================] - 1s 922us/step - loss: 1.2173 - accuracy: 0.6719 - val_loss: 1.7677 - val_accuracy: 0.4740
Epoch 4/45
990/990 [==============================] - 1s 887us/step - loss: 1.1820 - accuracy: 0.6676 - val_loss: 1.7713 - val_accuracy: 0.4683
Epoch 5/45
990/990 [==============================] - 1s 889us/step - loss: 1.1778 - accuracy: 0.6679 - val_loss: 1.7573 - val_accuracy: 0.4751
Epoch 6/45
990/990 [==============================] - 1s 920us/step - loss: 1.1509 - accuracy: 0.6737 - val_loss: 1.7670 - val_accuracy: 0.4744


In [512]:
survey_predictions = model_stacked_w.predict(survey_predict)
predicted_classes = np.argmax(survey_predictions, axis=1)
original_classes=survey.main_label.map(list(class_names).index) 
print(classification_report(original_classes, predicted_classes, target_names=class_names,zero_division=0))
prediction_score(original_classes,survey_predictions)

                       precision    recall  f1-score   support

            Biography       0.43      0.60      0.50         5
            Childrens       1.00      0.60      0.75         5
              Fantasy       0.50      0.80      0.62         5
           Historical       0.38      0.60      0.46         5
               Horror       0.25      0.20      0.22         5
      Mystery & Crime       0.50      0.20      0.29         5
               Poetry       0.40      0.40      0.40         5
   Politics & History       1.00      0.20      0.33         5
Religion & Philosophy       0.57      0.80      0.67         5
              Romance       0.57      0.80      0.67         5
      Science Fiction       0.60      0.60      0.60         5
       Sequential Art       1.00      0.80      0.89         5
          Young Adult       0.75      0.60      0.67         5

             accuracy                           0.55        65
            macro avg       0.61      0.55      0.54 